# Resistance/Support AND Candles Patterns

In [1]:
import pandas as pd
df = pd.read_csv("Data\META_4000_days.csv")

#Check if NA values are in data
df=df[df['Volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
2750,2024-04-02,154.750000,155.990005,153.460007,155.869995,155.869995,17598100
2751,2024-04-03,154.919998,156.550003,154.132004,156.369995,156.369995,17266200
2752,2024-04-04,155.080002,156.179993,151.880005,151.940002,151.940002,24184800
2753,2024-04-05,151.679993,154.839996,151.080994,153.940002,153.940002,16289800
2754,2024-04-08,153.990005,156.630005,153.990005,156.380005,156.380005,5566158


# Support and Resistance FUNCTIONS

In [2]:
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.Low[i]>df1.Low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.Low[i]<df1.Low[i-1]):
            return 0
    return 1

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.High[i]<df1.High[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.High[i]>df1.High[i-1]):
            return 0
    return 1

In [3]:
length = len(df)
high = list(df['High'])
low = list(df['Low'])
close = list(df['Close'])
open = list(df['Open'])
bodydiff = [0] * length

highdiff = [0] * length
lowdiff = [0] * length
ratio1 = [0] * length
ratio2 = [0] * length

def isEngulfing(l):
    row=l
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001      

    bodydiffmin = 0.002
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]<close[row-1] and
        open[row]>close[row] and 
        (open[row]-close[row-1])>=-0e-5 and close[row]<open[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]>close[row-1] and
        open[row]<close[row] and 
        (open[row]-close[row-1])<=+0e-5 and close[row]>open[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0
       
def isStar(l):
    bodydiffmin = 0.0020
    row=l
    highdiff[row] = high[row]-max(open[row],close[row])
    lowdiff[row] = min(open[row],close[row])-low[row]
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001
    ratio1[row] = highdiff[row]/bodydiff[row]
    ratio2[row] = lowdiff[row]/bodydiff[row]

    if (ratio1[row]>1 and lowdiff[row]<0.2*highdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]>close[row]):
        return 1
    elif (ratio2[row]>1 and highdiff[row]<0.2*lowdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]<close[row]):
        return 2
    else:
        return 0
    
def closeResistance(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.High[l]-min(levels, key=lambda x:abs(x-df.High[l])))<=lim
    c2 = abs(max(df.Open[l],df.Close[l])-min(levels, key=lambda x:abs(x-df.High[l])))<=lim
    c3 = min(df.Open[l],df.Close[l])<min(levels, key=lambda x:abs(x-df.High[l]))
    c4 = df.Low[l]<min(levels, key=lambda x:abs(x-df.High[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0
    
def closeSupport(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.Low[l]-min(levels, key=lambda x:abs(x-df.Low[l])))<=lim
    c2 = abs(min(df.Open[l],df.Close[l])-min(levels, key=lambda x:abs(x-df.Low[l])))<=lim
    c3 = max(df.Open[l],df.Close[l])>min(levels, key=lambda x:abs(x-df.Low[l]))
    c4 = df.High[l]>min(levels, key=lambda x:abs(x-df.Low[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0

In [4]:
n1=2
n2=2
backCandles=30
signal = [0] * length

for row in range(backCandles, len(df)-n2):
    ss = []
    rr = []
    for subrow in range(row-backCandles+n1, row+1):
        if support(df, subrow, n1, n2):
            ss.append(df.Low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.High[subrow])
    #!!!! parameters
    if ((isEngulfing(row)==1 or isStar(row)==1) and closeResistance(row, rr, 150e-5) ):#and df.RSI[row]<30
        signal[row] = 1
    elif((isEngulfing(row)==2 or isStar(row)==2) and closeSupport(row, ss, 150e-5)):#and df.RSI[row]>70
        signal[row] = 2
    else:
        signal[row] = 0



In [5]:
df['signal']=signal

In [6]:
df[df['signal']==2].count()

Date         32
Open         32
High         32
Low          32
Close        32
Adj Close    32
Volume       32
signal       32
dtype: int64

In [7]:

df=df.iloc[1000:2000]
df

,Date,Open,High,Low,Close,Adj Close,Volume,signal
1000,2017-04-18,41.710999,41.946499,41.635502,41.841000,41.841000,16734000,0
1001,2017-04-19,41.989498,42.111000,41.814499,41.910500,41.910500,19084000,0
1002,2017-04-20,42.071999,42.259998,41.966000,42.082500,42.082500,19180000,0
1003,2017-04-21,42.144001,42.194000,42.029999,42.159500,42.159500,26472000,0
1004,2017-04-24,42.560001,43.172501,42.493000,43.138000,43.138000,27450000,0
...,...,...,...,...,...,...,...,...
1995,2021-03-31,102.956001,104.666351,102.837250,103.431503,103.431503,29198000,0
1996,2021-04-01,104.897499,107.147003,104.844498,106.887497,106.887497,33980000,0
1997,2021-04-05,107.647003,111.865501,107.581001,111.277496,111.277496,43298000,0
1998,2021-04-06,111.125000,111.883003,110.739998,111.237503,111.237503,27060000,0


In [8]:
def SIGNAL():
    return df.signal

In [9]:
#A new strategy needs to extend Strategy class and override its two abstract methods: init() and next().
#Method init() is invoked before the strategy is run. Within it, one ideally precomputes in efficient, 
#vectorized manner whatever indicators and signals the strategy depends on.
#Method next() is then iteratively called by the Backtest instance, once for each data point (data frame row), 
#simulating the incremental availability of each new full candlestick bar.

#Note, backtesting.py cannot make decisions / trades within candlesticks — any new orders are executed on the
#next candle's open (or the current candle's close if trade_on_close=True). 
#If you find yourself wishing to trade within candlesticks (e.g. daytrading), you instead need to begin 
#with more fine-grained (e.g. hourly) data.

In [20]:
from backtesting import Strategy

class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        if self.signal1==2:
            sl1 = self.data.Close[-1] - 2
            tp1 = self.data.Close[-1] + 4
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==1:
            sl1 = self.data.Close[-1] + 2
            tp1 = self.data.Close[-1] - 4
            self.sell(sl=sl1, tp=tp1)

In [21]:
from backtesting import Backtest

bt = Backtest(df, MyCandlesStrat, cash=10_000, commission=.002)
stat = bt.run()
stat



C:\Users\Asus\AppData\Local\Temp\ipykernel_2964\3988754407.py:3: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, MyCandlesStrat, cash=10_000, commission=.002)


Start                                  1000.0
End                                    1999.0
Duration                                999.0
Exposure Time [%]                        30.7
Equity Final [$]                 11482.821672
Equity Peak [$]                  11767.504404
Return [%]                          14.828217
Buy & Hold Return [%]              168.836792
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -19.620673
Avg. Drawdown [%]                   -3.978688
Max. Drawdown Duration                  481.0
Avg. Drawdown Duration                   80.3
# Trades                                 22.0
Win Rate [%]                        40.909091
Best Trade [%]                       8.220546
Worst Trade [%]                     -6.293074
Avg. Trade [%]                    

In [22]:
bt.plot()

c:\Users\Asus\anaconda3\envs\algo_trading\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\Users\Asus\anaconda3\envs\algo_trading\lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2027', ...)